In [1]:
import requests
import pandas as pd
import io
import get_data_from_atlas as get_atlas

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
topics_dic = {"Low food access":"LFA",
"Firearm-related homicide mortality rate":"VRFIR",
"Food stamps (SNAP)":"SNP",
"Hardship Index":"HDX",
"Homicide mortality rate":"VRHOR",
"Neighborhood safety rate":"HCSNSP",
"Per capita income":"PCI",
"Poverty rate":"POV"}

In [2]:
topics_dic = {"Population":"POP"}

In [17]:
period_dic = {"2014-2018":"2014-2018",
              "2015":"2015",
              "2015-2017":"2015-2017",
              "2015-2019":"2015-2019",
              "2016-2018":"2016-2018",
              "2019":"2019"
              }

In [3]:
period_dic = {"POP2019":"2015-2019"}

In [19]:
def get_atlas_ver2(topics_dic, period_dic):
    cols = ['key', 'period', 'data_value', 'geo_layer', 'geo_id_label', 'population', 'std_error']
    df = pd.DataFrame(columns = cols)
    for _, topic in topics_dic.items():
        for _, period in period_dic.items():
            params = {"topic":topic, "population":"", "period": period, "layer":"neighborhood"}
            #params = {"topic":topic, "population":"", "period": period, "layer":"place"}
            #params = {"topic":topic, "population":"", "period": period, "layer":"zip"}
            url = "https://chicagohealthatlas.org/api/v1/data"
            data = requests.get(url, params=params)
            results = data.json()["results"]
            df_results = get_atlas.append_results(results)
            #print(df_results)
            df = pd.concat([df, df_results], axis=0)
    df = post_processing(df)
    return df
    #df.to_csv("atlas_result.csv", index=False)

In [20]:
def post_processing(df):
    key_name_dic =  create_key_name_dic()
    for _, row in df.iterrows():
        row_key = row['key']
        row['key'] = key_name_dic[row_key]

    df['geo_id_label'] = df['geo_id_label'].str.replace('1714000-','')
    #df = df.loc[:, ['key','geo_id_label','period','data_value']]
    df_result = key_to_row(df)

    return df_result

In [21]:
def key_to_row(df):
    name_list = []
    for _, row in df.iterrows():
        name_list.append(row['key'])
    name_list = list(set(name_list))

    df2 = pd.pivot(df, index=['geo_id_label', 'period'], columns='key', values='data_value')
    df2 = df2[name_list].reset_index()
    
    return df2

In [22]:
def create_key_name_dic():
    dic = {}
    key_name_list = pd.read_csv("atlas_key_list.csv")
    for _, row in key_name_list.iterrows():
        dic[row['key']] = row['name']
    return dic

In [23]:
get_atlas_ver2(topics_dic, period_dic)

/home/tnitta/capp30122/proj-food_in_the_hood/data/get_data_from_atlas.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'key': key, 'period': period, 'data_value': data_value, 'geo_layer': geo_layer,
/home/tnitta/capp30122/proj-food_in_the_hood/data/get_data_from_atlas.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'key': key, 'period': period, 'data_value': data_value, 'geo_layer': geo_layer,
/home/tnitta/capp30122/proj-food_in_the_hood/data/get_data_from_atlas.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'key': key, 'period': period, 'data_value': data_value, 'geo_layer': geo_layer,
/home/tnitta/capp30122/proj-food_in_the_hood/data/get_data_from_atlas.py:

key,geo_id_label,period,Population
0,1,2015-2019,52553.206784
1,10,2015-2019,38676.029693
2,11,2015-2019,27475.359833
3,12,2015-2019,19361.55814
4,13,2015-2019,18896.628476
...,...,...,...
72,75,2015-2019,21594.815938
73,76,2015-2019,15655.21018
74,77,2015-2019,48691.025555
75,8,2015-2019,84576.024332


In [24]:
def concatenating_datasets():
    df_crime = pd.read_csv('crime/rolling_total_crime.csv')
    
    df_pop = pd.read_csv('total_population.csv')
    df_pop = df_pop.rename(columns={'period':'year'})
    df_pop = df_pop.rename(columns={'geo_id_label':'community_area'})
    df_pop = df_pop.rename(columns={'data_value':'population'})

    df_merge = pd.merge(df_crime, df_pop, on=['community_area','year'])

    df_merge['crime_rate'] = (df_merge['crime_num']/df_merge['population'])*10000

    df_atlas = pd.read_csv('atlas_result.csv')
    df_atlas = df_atlas.rename(columns={'period':'year'})
    df_atlas = df_atlas.rename(columns={'geo_id_label':'community_area'})
    df_atlas = df_atlas.rename(columns={'data_value':'population'})

    df_concatenate = pd.merge(df_atlas, df_merge ,on=['community_area','year'], how='left')
    df_concatenate = df_concatenate.sort_values(['community_area', 'year'], ignore_index=True)

    df_concatenate.to_csv("food_and_hood_data.csv", index=False)

In [19]:
df_crime = pd.read_csv('crime/rolling_total_crime.csv')

df_pop = pd.read_csv('total_population.csv')
df_pop = df_pop.rename(columns={'period':'year'})
df_pop = df_pop.rename(columns={'geo_id_label':'community_area'})
df_pop = df_pop.rename(columns={'data_value':'population'})

df_merge = pd.merge(df_crime, df_pop, on=['community_area','year'])

df_merge['crime_rate'] = (df_merge['crime_num']/df_merge['population'])*10000

In [68]:
df_atlas = pd.read_csv('atlas_result.csv')
df_atlas = df_atlas.rename(columns={'period':'year'})
df_atlas = df_atlas.rename(columns={'geo_id_label':'community_area'})
df_atlas = df_atlas.rename(columns={'data_value':'population'})

In [21]:
df_concatenate = pd.merge(df_atlas, df_merge ,on=['community_area','year'], how='left')

In [26]:
df_concatenate = df_concatenate.sort_values(['community_area', 'year'], ignore_index=True)

In [30]:
concatenating_datasets()

In [21]:
df_crime = pd.read_csv('crime/rolling_total_crime.csv')

df_pop = pd.read_csv('total_population.csv')
df_pop = df_pop.rename(columns={'period':'year'})
df_pop = df_pop.rename(columns={'geo_id_label':'community_area'})
df_pop = df_pop.rename(columns={'data_value':'population'})

df_merge = pd.merge(df_crime, df_pop, on=['community_area','year'])

df_merge['crime_rate'] = (df_merge['crime_num']/df_merge['population'])*10000

df_merge

,community_area,year,crime_num,population,crime_rate
0,1,2015-2019,3816.8,52553.206784,726.273473
1,2,2015-2019,3396.6,78444.202314,432.995671
2,3,2015-2019,3658.8,53663.317254,681.806528
3,4,2015-2019,1923.0,42033.841869,457.488517
4,5,2015-2019,1422.8,35603.029202,399.628917
...,...,...,...,...,...
149,73,2014-2018,3187.6,27247.601967,1169.864417
150,74,2014-2018,613.2,19348.889650,316.917410
151,75,2014-2018,2105.2,22225.163841,947.214615
152,76,2014-2018,1785.4,15414.729559,1158.242831


In [22]:
df_atlas = pd.read_csv('atlas_result.csv')
df_atlas = df_atlas.rename(columns={'period':'year'})
df_atlas = df_atlas.rename(columns={'geo_id_label':'community_area'})
df_atlas = df_atlas.rename(columns={'data_value':'population'})

df_atlas

,Unnamed: 0,key,community_area,year,population
0,0,Firearm-related homicide mortality rate,35,2015-2019,21.800000
1,1,Firearm-related homicide mortality rate,36,2015-2019,45.300000
2,2,Firearm-related homicide mortality rate,37,2015-2019,88.700000
3,3,Firearm-related homicide mortality rate,38,2015-2019,33.300000
4,4,Firearm-related homicide mortality rate,39,2015-2019,13.300000
...,...,...,...,...,...
842,72,Poverty rate,74,2014-2018,4.146138
843,73,Poverty rate,75,2014-2018,19.109673
844,74,Poverty rate,76,2014-2018,13.501852
845,75,Poverty rate,77,2014-2018,14.576075


In [84]:
#url = "https://chicagohealthatlas.org/api/v1/data/?topic=POP&population=H&period=2014-2018&layer=zip"
#url = "https://chicagohealthatlas.org/api/v1"
cols = ['key', 'period', 'data_value', 'geo_layer', 'geo_id_label', 'population', 'std_error']
df = pd.DataFrame(columns = cols)
url = "https://chicagohealthatlas.org/api/v1/data"
params = {"topic":"LFA", "population":"", "period":"2019", "layer":"zip"}
data = requests.get(url, params=params)
results = data.json()["results"]



In [ ]:
url = "https://metopio.blob.core.windows.net/lalage/atlas/3/shapes/communityareas.topo.json"
data = requests.get(url)
results = data.json()
results['objects']
geo = results['objects']['features']['geometries']
for g in geo:
    print(g['id'])
    print(g['properties']['name'])

In [ ]:
'''
        {
            "name": "Low food access",
            "key": "LFA",
            "description": "Percent of residents who have low access to food, defined solely by distance: further than 1/2 mile from the nearest supermarket in an urban area, or further than 10 miles in a rural area.",
            "units": "% of residents",
            "format": "2 decimal places",
            "keywords": "food insecurity",
            "is_count": false,
            "sources": [
                {
                    "notes": "Food Access Research Atlas",
                    "source": {
                        "name": "USDA, Economic Research Service, Food Access Research Atlas",
                        "label": "usda"
                    }
                }
            ],
            "subcategories": [
                {
                    "name": "Resource Availability",
                    "slug": "resource-availability",
                    "category": "Physical Environment"
                }
            ],
            "restrictions": []
        },
'''